In [1]:
# import the necessary packages
import pandas as pd
import pickle
import datetime
from dateutil import parser
import re

In [2]:
data = pd.read_pickle('ds-spring-2017/pickled_data.p')

text_raw = data['description']

#to lowercase
text_lower = [s.lower() for s in text_raw]

#remove any of below characters and replace with space - excesses will be removed later
text_lower = [re.sub('[#!?*";,())]',' ',s) for s in text_lower]

#remove below characters only if they are not preceded by a number
text_lower = [re.sub('((?<!\d)[.:-])',' ',s) for s in text_lower]

#insert spaces between letters and numbers where 
text_lower = [re.sub(r'((?<=[a-z])\d)',r' \1',s) for s in text_lower]
text_lower = [re.sub(r'((?<=\d)[a-z])',r' \1',s) for s in text_lower]

#remove all html tags
text_lower = [re.sub('(<.*?>)',' ',s) for s in text_lower]

#replace ampersand with and
text_lower = [re.sub('[&]',' and ',s) for s in text_lower]

#replace w/ with "with"
text_lower = [re.sub('w/',' with ',s) for s in text_lower]

#replace / symbols with space
text_lower = [re.sub('/',' ',s) for s in text_lower]

#remove text artifacts
text_lower = [re.sub('website_redacted',' ',s) for s in text_lower]
text_lower = [re.sub('\xa0',' ',s) for s in text_lower]
text_lower = [re.sub('\r','',s) for s in text_lower]
text_lower = [re.sub('\t','',s) for s in text_lower]
text_lower = [re.sub('_','',s) for s in text_lower]

#replace excess spaces
text_lower = [re.sub(' +',' ',s).strip() for s in text_lower]

#adjust important abbreviations
text_lower = [re.sub(' br ',' bedroom ',s) for s in text_lower]
text_lower = [re.sub('sq ft','square feet',s) for s in text_lower]
text_lower = [re.sub(' ss ',' stainless steel',s) for s in text_lower]
text_lower = [re.sub(' s s ',' staineless steel ',s) for s in text_lower]

#remove trailing "<a"
text_cleaned = [re.sub('<a','',s).strip() for s in text_lower]


In [3]:
data['cleantext'] = text_cleaned
data.drop('description',axis=1)
print('done')

done


In [4]:
X = data.drop(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'street_address', 'features', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'manager_id', 'photos'],axis=1)
Y = data['interest_level']

In [5]:
# Y[(Y=='low')]=1
# Y[(Y=='medium')]=2
# Y[(Y=='high')]=3
# Y.head()

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8)

In [8]:
binary_vectorizer = CountVectorizer(binary=True,stop_words='english')
binary_vectorizer.fit(X_train['cleantext'])

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [9]:
# tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_vectorizer.fit(X_train['cleantext'])

In [10]:
train_bin_vector = binary_vectorizer.transform(X_train['cleantext'])
test_bin_vector = binary_vectorizer.transform(X_test['cleantext'])

In [11]:
X_train_bin = X_train
X_train_bin['textvector'] = train_bin_vector
X_train_bin.drop('cleantext',axis=1)

X_test_bin = X_test
X_test_bin['textvector'] = test_bin_vector
X_test_bin.drop('cleantext',axis=1)
print('done')

done


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
l_model_binary = LogisticRegression()
l_model_binary.fit(train_bin_vector, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
# train_tfidf_vector = tfidf_vectorizer.transform(X_train['cleantext'])
# test_tfidf_vector = tfidf_vectorizer.transform(X_test['cleantext'])

In [14]:
# X_train_tfidf = X_train
# X_train_tfidf['textvector'] = train_tfidf_vector
# X_train_tfidf.drop('cleantext',axis=1)
# print('done')

In [15]:
# l_model_tfidf = LogisticRegression()
# l_model_tfidf.fit(X_train_tfidf, Y_train)

In [17]:
l_model_binary.predict(test_bin_vector)

array(['low', 'low', 'low', ..., 'medium', 'low', 'low'], dtype=object)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 7

ValueError: Data is not binary and pos_label is not specified